In [1]:
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import dill
import random

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [4]:
TRAIN_DATASET_PATH = '/content/train.csv'
TEST_DATASET_PATH = '/content/test.csv'

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры


In [5]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.tail()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
9995,1260,61,2.0,49.090728,33.272626,6.0,3,12.0,1981,0.300323,B,B,52,10311,6,NaN,1,9,B,119367.455796
9996,16265,27,2.0,64.307684,37.038420,9.0,13,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,199715.148807
9997,2795,178,1.0,29.648057,16.555363,5.0,3,5.0,1958,0.460556,B,B,20,4386,14,NaN,1,5,B,165953.912580
9998,14561,21,1.0,32.330292,22.326870,5.0,3,9.0,1969,0.194489,B,B,47,8004,3,125.0,3,5,B,171842.411855
9999,7202,94,1.0,35.815476,22.301367,6.0,9,9.0,1975,0.127376,B,B,43,8429,3,NaN,3,9,B,177685.627486


**Задача:** разработать модель, которая будет предсказывать цену на недвижимость.

In [6]:
# Данные без целевого показателя.
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
4995,10379,29,2.0,43.177521,30.339945,5.0,6,5.0,1962,0.069660,B,B,31,6119,4,NaN,1,2,B
4996,16138,38,3.0,93.698122,94.521465,10.0,21,27.0,2018,0.060753,B,B,15,2787,2,520.0,0,7,B
4997,3912,101,1.0,33.656723,19.003259,5.0,2,5.0,1966,0.038693,B,B,28,6533,1,1015.0,2,5,B
4998,5722,10,1.0,38.635155,20.976257,9.0,8,14.0,1970,0.089040,B,B,33,7976,5,NaN,0,11,B
4999,11004,21,2.0,67.122742,33.944344,13.0,9,17.0,2009,0.194489,B,B,47,8004,3,125.0,3,5,B


# **Разделяем данные на train и test и сохраним на диск**

In [7]:
X = train_df.drop(columns='Price')
y = train_df['Price']

In [8]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians = None
        self.kitchen_square_quantile = None
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
    
    def transform(self, X):
        """Трансформация данных"""

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms']
          # KitchenSquare
        condition = (X['KitchenSquare'].isna()) \
                    | (X['KitchenSquare'] > self.kitchen_square_quantile)
        
        X.loc[condition, 'KitchenSquare'] = self.medians['KitchenSquare']

        X.loc[X['KitchenSquare'] < 3, 'KitchenSquare'] = 3
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']

        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor'] = X['Floor']
        
        # HouseYear
        current_year = datetime.now().year
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # Healthcare_1
        if 'Healthcare_1' in X.columns:
            X.drop('Healthcare_1', axis=1, inplace=True)
            
        # LifeSquare
        X['LifeSquare_nan'] = X['LifeSquare'].isna() * 1
        condition = (X['LifeSquare'].isna()) & \
                      (~X['Square'].isna()) & \
                      (~X['KitchenSquare'].isna())
        
        X.loc[condition, 'LifeSquare'] = X.loc[condition, 'Square'] - X.loc[condition, 'KitchenSquare'] - 3
        
        
        X.fillna(self.medians, inplace=True)
        
        return X

In [9]:
train_df.isna().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=21)

# save test

X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train

X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

# **Выбираем показатели и собираем pipeline**



Выбираем показатели, по которым будем строить модель, заведем новые показатели, которые могут коррелируются с целевым показателем 

In [40]:
feature_names = ['Rooms', 'Square', 'KitchenSquare']


target_name = 'Price'

Соберем кусок, ответственный за feature engineering

In [43]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.key]]

In [44]:
#combine
Rooms_tr = []
Square_tr = []
KitchenSquare_tr = []


Rooms_t = Pipeline([
                 ('selector', NumberSelector(key='Rooms'))
            ])
Rooms_tr.append(('Rooms', Rooms_t))
Square_t = Pipeline([
                 ('selector', NumberSelector(key='Square'))
            ])
Square_tr.append(('Square', Square_t))
KitchenSquare_t = Pipeline([
                 ('selector', NumberSelector(key='KitchenSquare'))
            ])
KitchenSquare_tr.append(('KitchenSquare', KitchenSquare_t))
feats = FeatureUnion(Rooms_tr + Square_tr + KitchenSquare_tr)
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[ 1.        , 43.38079757,  1.        ],
       [ 3.        , 83.5963648 ,  1.        ],
       [ 2.        , 53.61058324,  8.        ],
       ...,
       [ 2.        , 45.05389835,  5.        ],
       [ 1.        , 40.33798661,  8.        ],
       [ 3.        , 63.54359087,  6.        ]])

Добавляем модель в pipeline

In [49]:
%%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression())
])

CPU times: user 15 µs, sys: 3 µs, total: 18 µs
Wall time: 19.6 µs


In [50]:
classifier.fit(X_train, y_train)

ValueError: ignored

In [22]:
# кроссвалидация
cv_scores = cross_val_score(classifier, X_train, y_train, cv=6)
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print ('CV score is {}+-{}'.format(cv_score,cv_score_std))

# обучение pipeline на тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict (X_test)[:,1]

ValueError: ignored

In [51]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Rooms',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='Rooms'))])),
                                 ('Square',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='Square')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('KitchenSquare',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='KitchenSquare')),
                                                  ('standard',
                                                   StandardScaler())]))])),
 ('classifier', LogisticRegression(C=10, random_state=42))]

In [52]:
## Сохраним модель (пайплайн)

with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

# **Flask**

In [53]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

In [55]:
cat /root/.ngrok2/ngrok.yml

cat: /root/.ngrok2/ngrok.yml: No such file or directory


In [ ]:
class ClientDataForm(FlaskForm):
   Rooms = StringField('Rooms', validators=[DataRequired()])
   Square = StringField('Square', validators=[DataRequired()])
   KitchenSquare = StringField('KitchenSquare', validators=[DataRequired()])

In [ ]:
app = Flask(__name__)
app.config.update(
    CSRF_ENABLED=True,
    SECRET_KEY='you-will-never-guess',
)

In [ ]:
def get_prediction(Rooms, Square, KitchenSquare):
    body = {'Rooms': Rooms,
            'Square': Square,
            'KitchenSquare': KitchenSquare}

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [ ]:
@app.route("/")
def index():
    return render_template('index.html')

In [ ]:
@app.route('/predicted/<response>')
def predicted(response):
    response = json.loads(response)
    print(response)
    return render_template('predicted.html', response=response)

In [ ]:
@app.route('/predict_form', methods=['GET', 'POST'])
def predict_form():
    form = ClientDataForm()
    data = dict()
    if request.method == 'POST':
        data['Rooms'] = request.form.get('Rooms')
        data['Square'] = request.form.get('Square')
        data['KitchenSquare'] = request.form.get('KitchenSquare')


        try:
            response = str(get_prediction(data['Rooms'],
                                      data['Square'],
                                      data['KitchenSquare'])

            print(response)
        except ConnectionError:
            response = json.dumps({"error": "ConnectionError"})
        return redirect(url_for('predicted', response=response))
    return render_template('form.html', form=form)']))
            print(response)
        except ConnectionError:
            response = json.dumps({"error": "ConnectionError"})
        return redirect(url_for('predicted', response=response))
    return render_template('form.html', form=form)

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8181, debug=True)